In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

# --- 설정 ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 256
BATCH_SIZE = 16 
EPOCHS = 5
NUM_CLASSES = 531
HIDDEN_DIM = 768

# 경로
BASE_DIR = "Amazon_products"
TRAIN_CORPUS_PATH = os.path.join(BASE_DIR, "train/train_corpus.txt")
TEST_CORPUS_PATH = os.path.join(BASE_DIR, "test/test_corpus.txt")
HIERARCHY_PATH = os.path.join(BASE_DIR, "class_hierarchy.txt")
OUTPUT_DIR = "saved_ensemble"

# 사용할 데이터 파일들
SILVER_FILE = "train_round_2.csv"
GOLD_FILE_1 = "gold_labels.csv"
GOLD_FILE_2 = "gold_labels2.csv"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# --- 데이터 로드 및 병합 ---
def load_merged_data():
    print("1. Merging Data & Cleaning...")
    id2text = {}
    
    # ID 정제 함수
    def clean_id(x):
        try: return str(int(float(x)))
        except: return str(x).strip()

    # 텍스트 로드
    loaded_files = 0
    for path in [TRAIN_CORPUS_PATH, TEST_CORPUS_PATH]:
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    parts = line.strip().split("\t", 1)
                    if len(parts) == 2:
                        idx = clean_id(parts[0])
                        id2text[idx] = parts[1]
            loaded_files += 1
    
    if loaded_files == 0:
        raise FileNotFoundError("❌ Error: corpus files not found.")

    # CSV 로드
    dfs = []
    input_files = [SILVER_FILE, GOLD_FILE_1, GOLD_FILE_2]
    
    for filepath in input_files:
        if os.path.exists(filepath):
            try:
                df = pd.read_csv(filepath)
                # 컬럼명 표준화
                df.columns = [c.strip().lower() for c in df.columns]
                if 'pid' in df.columns: df.rename(columns={'pid': 'id'}, inplace=True)
                
                if 'id' not in df.columns or 'labels' not in df.columns:
                    print(f"   ⚠️ Skipping {filepath}: columns missing")
                    continue

                df['id'] = df['id'].apply(clean_id)
                
                dfs.append(df)
            except Exception as e:
                print(f"   ⚠️ Error reading {filepath}: {e}")
        else:
            if filepath == SILVER_FILE:
                raise FileNotFoundError(f"❌ Error: {SILVER_FILE} not found.")

    df_final = pd.concat(dfs, ignore_index=True)
    
    # [핵심] 라벨 데이터 클리닝 (None, NaN, 문자열 등 제거)
    def is_valid_label(x):
        s = str(x).strip().lower()
        if s in ['none', 'nan', '', 'null']: return False
        # 숫자가 하나라도 있어야 함
        return any(c.isdigit() for c in s)

    # 불량 라벨 제거
    initial_len = len(df_final)
    df_final = df_final[df_final['labels'].apply(is_valid_label)]
    cleaned_len = len(df_final)
    if initial_len != cleaned_len:
        print(f"   🧹 Cleaned {initial_len - cleaned_len} invalid label rows.")

    # 텍스트 매핑 & 결측치 제거
    df_final['text'] = df_final['id'].map(id2text)
    df_final = df_final.dropna(subset=['text', 'labels'])
    
    print(f"   ✅ Final Training Size: {len(df_final)}")
    return df_final

# --- 통합 모델 클래스 ---
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias: self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else: self.register_parameter('bias', None)
        self.reset_parameters()
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None: self.bias.data.uniform_(-stdv, stdv)
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias if self.bias is not None else output

class GNN_Model(nn.Module):
    def __init__(self, model_name, num_classes, hidden_dim, adj_matrix, model_type="bert"):
        super(GNN_Model, self).__init__()
        self.model_type = model_type
        self.encoder = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.label_embedding = nn.Parameter(torch.FloatTensor(num_classes, hidden_dim))
        nn.init.xavier_uniform_(self.label_embedding)
        self.gcn = GraphConvolution(hidden_dim, hidden_dim)
        self.adj_matrix = adj_matrix

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        
        # 모델 타입에 따라 임베딩 추출 방식 분기 처리
        if self.model_type == "bert":
            doc_embedding = outputs.pooler_output
        else: # deberta
            doc_embedding = outputs.last_hidden_state[:, 0, :]
            
        doc_embedding = self.dropout(doc_embedding)
        refined_label = torch.tanh(self.gcn(self.label_embedding, self.adj_matrix))
        return torch.mm(doc_embedding, refined_label.t())

# --- 학습 함수 ---
def train_one_model(model_name, save_name, df, adj_matrix, model_type):
    print(f"\n🚀 Training Model: {model_name} ({model_type})")
    
    # Tokenizer (use_fast=False 필수)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    
    class ReviewDataset(Dataset):
        def __init__(self, texts, labels):
            self.texts = texts
            self.labels = labels
        def __len__(self): return len(self.texts)
        def __getitem__(self, item):
            lbls = [int(x) for x in str(self.labels[item]).split(",") if x]
            lbl_tensor = torch.zeros(NUM_CLASSES)
            lbl_tensor[lbls] = 1.0
            enc = tokenizer.encode_plus(str(self.texts[item]), padding='max_length', 
                                      truncation=True, max_length=MAX_LEN, return_tensors='pt')
            return {'ids': enc['input_ids'].flatten(), 'mask': enc['attention_mask'].flatten(), 'lbl': lbl_tensor}

    # Dataset & Loader
    dataset = ReviewDataset(df.text.to_numpy(), df.labels.to_numpy())
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    
    # Model Init
    model = GNN_Model(model_name, NUM_CLASSES, HIDDEN_DIM, adj_matrix, model_type).to(DEVICE)
    optimizer = AdamW(model.parameters(), lr=1e-5) # 학습률 통일
    scheduler = get_linear_schedule_with_warmup(optimizer, 0, len(loader) * EPOCHS)
    loss_fn = nn.BCEWithLogitsLoss()
    
    # Training Loop
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            ids, mask, lbl = batch['ids'].to(DEVICE), batch['mask'].to(DEVICE), batch['lbl'].to(DEVICE)
            optimizer.zero_grad()
            logits = model(ids, mask)
            loss = loss_fn(logits, lbl)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        print(f"   Loss: {total_loss / len(loader):.4f}")
    
    # Save
    save_path = os.path.join(OUTPUT_DIR, save_name)
    torch.save(model, save_path)
    print(f"✅ Model Saved: {save_path}")

# --- 메인 실행 ---
def main():
    # 1. 데이터 준비
    df = load_merged_data()
    
    # 2. Adjacency Matrix 생성
    adj = torch.eye(NUM_CLASSES)
    with open(HIERARCHY_PATH, "r") as f:
        for line in f:
            p, c = map(int, line.split())
            if p < NUM_CLASSES and c < NUM_CLASSES: adj[p,c]=1; adj[c,p]=1
    rowsum = torch.sum(adj, dim=1)
    d_inv_sqrt = torch.pow(rowsum, -0.5)
    d_inv_sqrt[torch.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = torch.diag(d_inv_sqrt)
    adj = torch.mm(torch.mm(d_mat_inv_sqrt, adj), d_mat_inv_sqrt).to(DEVICE)

    # 3. 순차 학습 진행
    # (1) BERT 모델 학습
    train_one_model("bert-base-uncased", "bert_final.pt", df, adj, "bert")
    
    # (2) DeBERTa 모델 학습
    train_one_model("microsoft/deberta-v3-base", "deberta_final.pt", df, adj, "deberta")

    print("\n🎉 All Training Completed! Ready for Ensemble Inference.")

if __name__ == "__main__":
    main()